In [1]:
from __future__ import print_function
import csv
import pickle
import numpy as np
from PIL import Image
import time
from keras.applications import*
import keras
from keras.models import Sequential
from keras.optimizers import SGD, Adam
from keras.layers import Dense, Dropout, Flatten
from keras.layers import *
from keras.models import Model
from keras.applications.inception_resnet_v2 import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from keras.utils import multi_gpu_model
import pickle
import keras.backend as K
from utils import *
import os, sys
from skll.metrics import spearman
from scipy.stats import pearsonr


%load_ext autoreload
%autoreload 2

Using TensorFlow backend.


### Set up

In [4]:
data_dir='kadid10k/images/'
input_shape = (512, 512, 3)
batch_size = 128
data_ids = pd.read_csv('kadid10k/dmos.csv')
print('data number:',len(data_ids))

data number: 10125


### Step 1: Extract MLSP feature from pre-trained model

In [5]:
model = model_inceptionresnet_multigap()
data_feats = extract_mlsp_feats(data_ids,model,data_dir)
data_dmos = data_ids[['dmos']].values

1000 images
2000 images
3000 images
4000 images
5000 images
6000 images
7000 images
8000 images
9000 images
10000 images
Done...


### Step 2: Train a regression model for IQA

In [9]:
iter_num = 5
srocc_result = []

for i in range(iter_num):
    print(str(i+1))
    data = sample_data(filepath='kadid10k/dmos.csv',feats=data_feats,dmos=data_dmos)
    model = fc_model()
    model.compile(loss=keras.losses.mean_squared_error,
                  optimizer=keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, 
                                                  epsilon=None, decay=0.0,amsgrad=False))
    # checkpoint
    filepath ='model/multigap_best_model.hdf5'
    checkpoint = keras.callbacks.ModelCheckpoint(filepath, 
                                                 monitor='val_loss', 
                                                 verbose=0, 
                                                 save_best_only=True, 
                                                 mode='min')
    callbacks_list = [checkpoint]
    
    history = model.fit(data['train_feats'],data['train_dmos'],
                        batch_size=batch_size,
                        epochs=30,
                        verbose=0,
                        callbacks=callbacks_list,
                        validation_data=(data['valid_feats'],data['valid_dmos']))
    model.load_weights(filepath)
    y_pred = model.predict(data['test_feats'], batch_size=batch_size)
    
    K.clear_session()
    
    test_srocc = spearman(data['test_dmos'],y_pred)
    print('srocc:', str(test_srocc))
    srocc_result.append(test_srocc)

1
srocc: 0.9366366703318774
2
srocc: 0.9251692421223607
3
srocc: 0.9256040742552997
4
srocc: 0.9207364475284746
5
srocc: 0.9420689617354758
